In [1]:
pip intall sklearn

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [2]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
# bla bla test

In [ ]:
# Laoading Data
data = pd.read_excel("3_individual_features_extended.xlsx", header = 0)
data.drop(columns = ["Unnamed: 0"], inplace = True)     # drop id column
shape_before = data.shape


In [6]:
data.head()

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,...,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
0,526,0.148336,7.850746,0.126865,56.210746,0.165958,451.54,0.264343,67,0.331683,...,4,5,7,8,7,2.57,-1.78,3.44,0.90,8.0
1,2648,0.746757,33.948718,0.548598,21.791169,0.064337,109.24,0.063952,78,0.386139,...,8,8,0,5,4,-0.24,-0.33,3.64,0.91,8.0
2,105,0.029611,8.076923,0.130520,57.649231,0.170205,141.72,0.082966,13,0.064356,...,8,6,3,2,6,-0.52,-2.03,3.98,0.90,8.0
3,162,0.045685,6.480000,0.104714,159.421600,0.470679,836.46,0.489685,25,0.123762,...,6,6,2,4,7,0.42,-2.04,4.22,0.90,8.0
4,105,0.029611,5.526316,0.089303,43.632632,0.128822,169.20,0.099054,19,0.094059,...,4,4,1,5,7,0.73,-1.48,3.74,0.91,8.0


In [3]:
data = data.apply(lambda col: col.astype(float) if col.dtype == 'int64' else col) # convert to float where possible
data = data.select_dtypes(include=['float64', 'float32'])                         # selecting only the float types
shape_after = data.shape

y = data["project"]                             # split response
X = data[data.columns.difference(['project'])]  # split predictor

In [ ]:
print(f"Number of Columns Removed due to type incompatability: {shape_before[1] - shape_after[1]}")
print("For these a levels system will need to be built in")

Number of Columns Removed due to type incompatability: 11


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data (important for Lasso, since it's sensitive to feature scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Lasso with a regularization parameter (alpha)
lasso = Lasso(alpha=0.1)  # You can tune alpha to see different levels of regularization
lasso.fit(X_train_scaled, y_train)

# Coefficients of the model
coefficients = lasso.coef_

# Display the non-zero features (selected features)
selected_features = np.where(coefficients != 0)[0]
print("Selected features:", selected_features)

# Number of features selected
print("Number of selected features:", len(selected_features))

# Optionally, you can put the feature importances into a DataFrame for clarity
feature_df = pd.DataFrame({
    'Feature': range(X.shape[1]),
    'Coefficient': coefficients
})
selected_feature_df = feature_df[feature_df['Coefficient'] != 0]
#print(selected_feature_df)

Selected features: [  0  13  18  21  25  44  48  63  64  69  76  77  79  83  86  99 105 111
 112 124]
Number of selected features: 20


In [11]:
data.columns[selected_features]

Index(['indiv_spoken_time', 'max_turns_without_speaking_ratio',
       'max_words_turn', 'speech_ang', 'text_anger', 'sentiment_avg',
       'ALTERNATIVE_REALITIES_Treehugger', 'LIFESTYLES_Fitness',
       'PERSONALITY_Journalist', 'EMOTIONS_Happy',
       'Behavioral_Values_Fairness_Unfairness',
       'Behavioral_Values_Fairness_Fairness', 'Groupflow_Leechflow',
       'EMOTIONS_ALL_Anger', 'EMOTIONS_ALL_Disgust', 'recreational_perceived',
       'A', 'purity_sanctity_score', 'dummy_question1', 'conservation'],
      dtype='object')